In [ ]:
!git clone https://github.com/rafaseto/PIBIC-AnaliseDados.git

In [ ]:
%cd PIBIC-AnaliseDados

In [ ]:
import pip

In [ ]:
!pip install dlisio

In [1]:
from dlisio import dlis
import pandas as pd

In [2]:
# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

In [3]:
dli = []        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
pocos = []      # Conterá strings identificadoras de 9 caracteres, tipo '3-CP-1847'
charId_inicio, charId_fim = 5, 14   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    pocos.append(file[charId_inicio : charId_fim])    # armazenando identificadores do tipo '3-CP-XXXX'
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    dli.append(leitura)         # armazenando os arquivos lógicos carregados com 'load'

In [ ]:
nomes = []
for poco in dli:
    nomes.append(poco.origins[0].well_name)
nomes

In [ ]:
# Criando um dicionário para associar nomes aos elementos de dli
dli_dict = {
    'p-47': dli[0],
    'p-48': dli[1],
    'p-51': dli[2],
    'p-53': dli[3],
    'p-55': dli[4],
    'p-57': dli[5]
}

In [ ]:
dli_dict['p-47'].describe()

In [ ]:
for poco in dli_dict.values():
    print(poco.origins[0].well_name)

In [ ]:
dli_dict['p-47'].channels

In [ ]:
dlis_df = []    # Conterá os dlis_df_dict respectivos aos poços

# Curvas de perfis que não serão utilizadas
curvas_nao_utilizadas = ['WF11', 'AHV', 'AHVT', 'BHV', 'BHVT', 'CS', 'FCPS', 'HDCN', 'HMCN', 'ITT', 'ITTT', 'LSPD', 'MMK', 'NCPS', 'SP', 'TENS']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for poco in dli_dict.values():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name not in curvas_nao_utilizadas    # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df.append(pd.DataFrame(curvas[curvas_utilizadas]))
    except:
        pass

In [ ]:
curvas_utilizadas

In [ ]:
# Criando um dicionário para associar nomes aos elementos de dlis_df
dlis_df_dict = {
    'p-47' : dlis_df[0],
    'p-48' : dlis_df[1],
    'p-51' : dlis_df[2],
    'p-53' : dlis_df[3],
    'p-55' : dlis_df[4],
    'p-57' : dlis_df[5]
}

In [ ]:
# Transformados os valores -999.25 em nulos
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

In [ ]:
from aux_datapreprocess import *

## Remoção de Colunas

In [ ]:
colunas_para_remover = ['CALI', 'RHOB', 'DRHO', 'PE', 'HDRS', 'HMRS', 'DTC']
dlis_df_dict['p-47'] = remove_colunas(dlis_df_dict['p-47'], colunas_para_remover)

In [ ]:
dlis_df_dict['p-47']

## Renomeação de Colunas

In [ ]:
dlis_df_dict['p-47'] = renomeiaColuna(dlis_df_dict['p-47'], "GR", "gamma-ray")

In [ ]:
dlis_df_dict['p-47']

## Transformação NPHI

In [ ]:
dlis_df_dict['p-53']

In [ ]:
nphi_transform(dlis_df_dict, ['p-47', 'p-53'])

In [ ]:
dlis_df_dict['p-53']

## Remoção de Pontos com Falha

calcular limites para DCALI (2 polegadas, se for maior descartar), DRHO (-0.15, +0.15) --> nesse intervalo, fora descarta

In [ ]:
dlis_df_dict['p-53']

In [ ]:
pontos_com_falha = {
    'p-47': [250, 360],
    'p-48': [75, 210],
    'p-51': [60, 260],
    'p-53': [90, 221],
    'p-55': [64, 215],
    'p-57': [90, 320]
}

In [ ]:
#dlis_df_dict['1847'].drop(dlis_df_dict['1847'][dlis_df_dict['1847']['TDEP'] > 300].index)

In [ ]:
# Chamando a função
remove_pontos_com_falha(dlis_df_dict, pontos_com_falha)

In [ ]:
add_DCALI(dlis_df_dict)

In [ ]:
limita_curva(dlis_df_dict, "DCALI", -2, 2)

In [ ]:
limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)

In [ ]:
pd.set_option('display.max_rows', None)
dlis_df_dict['p-53']